# Powered by Pandas
**Authors:** Raymond Tran, Edel Grace Altares<br/>
**Audience:** New programmers w/ a basic understanding of Python, procedural, and object-oriented programming.

Welcome to Powered by Pandas! This workbook is designed to introduce its audience to a little bit of the `pandas` and `matplotlib` Python libraries. The code is thoroughly documented and explained with section headings that give a general idea of what will be explained in a few chunks of code.

**Useful Things in Jupyter:**<br/>
- **Shift + Enter** allows you to execute the currently selected cell, and immediately moves the pointer to the next one for you.
- **Pressing Tab** immediately after a `.` in a line of code like `object.` will showcase all the methods/functions that can be used with that particular class. Note that `pandas` and `matplotlib` have *a lot*.

---
## Getting Started
When Jupyter is installed with Anaconda Navigator, many data analysis modules come with it.

Some other modules that won't be featured include:
- **seaborn**  (A prettier, fancier data visualizer with its own set of features.)
- **networkx** (Graph/Network modelling and manipulation.)
- **scipy**    (A scientific computing library for stuff like optimization, linear algebra, and so on.)
- **sklearn**  (A library for data classification and machine learning!)
- **numpy**    (A module that expands on the concept of arrays that are fast, efficient, and flexible. Numpy arrays are used in pandas internally.)

In [ ]:
# Import a module, but refer to it as something else in the code.
# For example, instead of calling pandas every time, just call it pd.
# And, let it know that you don't need to refer to the parent 'pd' every time when playing with DataFrames and Series.
# Without 'from pandas import DataFrame, Series' = pd.DataFrame.head
# With 'from pandas import DataFrame, Series' = Data.head

import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import numpy as np

---
## File IO, DataFrames, and You!

Let's start off by opening a small csv spreadsheet that contains some people, their favourite fruits, and what year in university they are!<br/>
To see what file-types are supported, you can check: [Pandas Doc - IO](https://pandas.pydata.org/pandas-docs/stable/io.html)

In [ ]:
# Quick Import.
quickDF = pd.read_clipboard()
quickDF

In [ ]:
# Create a DataFrame called csvDF.
# Then, fill it with the contents of the file.
csvFileName = "fruitsAndPeople.csv"
csvDF = pd.read_csv(csvFileName)
csvDF

**Dataframes are made up of a bunch of 'Series'** Similar to how matrices are made up of lists.<br/>
We can also add rows and columns of data, merge DataFrames, etc.<br/>
We'll show this by adding a new entry to the frame, as well as a new column.

In [ ]:
# Create a row of data. 
newRow1 = {'Name': 'Joe',
          'Fav Fruit': 'Watermelon', 
          'Yr. of Study': 4,
          'Plays Music': True}

# Create a new DataFrame, and put the row of data into it using append()
mergeDF = DataFrame()
# Ignore Index prevents duplicate index values by ensuring a new, unique one is assigned.
mergeDF = mergeDF.append(newRow1, ignore_index=True)

# We can also merge two dataframes like so:
csvDF = csvDF.append(mergeDF, ignore_index=True, sort=False)
#csvDF = csvDF[['Name','Fav Fruit','Yr. of Study','Plays Music']] # Sort the columns by how we know them. (Deprecated line.)
csvDF

In [ ]:
# Insert a column of data.
# Locate the row (nothing), and the column (Total).
# If the column doesn't exist, the DataFrame will create a new one and initialize it with the values.

csvDF.loc[:,"totlA"] = 18 # Oops, a mistake! Let's get rid of it.
csvDF.drop(columns=["totlA"], inplace = True) # The 'inplace' parameter saves the change onto the current object rather than returning.

csvDF.loc[:,"Zero"] = 0
csvDF

---
# Data Manipulation
Let's try importing a really big data file.
We can use DF.head() in order to print the first few entries.

`sales-full.csv` is a massive spreadsheet containing (dummy) data for six separate business men, and how much product they sold on each day.<br />
Let's how big this data set *really* is.

In [ ]:
# Import the data into a DataFrame.
salesDF = pd.read_csv("sales-full.csv")

# Check the shape (row x col)
print("Size of sales-full: " + str(salesDF.shape))


In [ ]:
# Instead of showing all 300 entries, we can just preview some of the contents with head(), tail(), and/or sample()
# You can pass an int argument to increase/decrease the entries shown. (Default 5)
x = 5

# Show the first x entries.
#salesDF.head(x)

# Show the last x entries.
#salesDF.tail(x)

# Show x randomly picked entries.
salesDF.sample(x)

**Let's fix up the column names for our data.** There are many ways to do this.<br/>
1. We can choose to just label them directly, but that means we have to label ALL of them.
2. Alternatively, we can get a copy of the column names, and index the column we want to change.
3. Finally, if we know the name of the column and just want to do some tweaking, we can use the built-in column renaming function.

In [ ]:
# Method 1
salesDF.columns = ["Day", "Alex", "Bruce", "Candice", "Denise", "Edel", "Frisk"]
salesDF.head(5)

# Method 2
salesDFColumns = salesDF.columns.values; # Produces a numpy array.
indexOfCandice = list(salesDFColumns).index("Candice") # Produces an ndarray, so we have to convert it to a list first, then get the index.
salesDFColumns[indexOfCandice] = "Carrie"
salesDF.head(5)

# Method 3
# We specify the previous name, and then the new name.
# The 'inplace' parameter ensures we get a new dataframe with the changes made.
salesDF.rename(columns = {'Bruce':'Brandon'}, inplace = True)
salesDF.head(5)

**Now, let's aggregate the data into a new column.**<br/>
This is a really big file, so we can't afford to do the math and manually insert it one by one.
This is where pandas come into play!

In [ ]:
# Locate the row (nothing), and the column (Total).
# If the column doesn't exist, the DataFrame will create a new one and initialize it with the values.
salesDF.loc[:,"Day Total"] = 0

salesDFShape = salesDF.shape
print("Shape of salesDF = ",salesDFShape)
salesDFColumns = salesDF.columns.values;

# For every row in salesDF...
for i in range(0, salesDFShape[0]):
    # Reset the total count.
    totalRowVal = 0
    # And for every column after 'Day'
    for j in salesDFColumns[1:]:
        # Aggregate the total across all sellers.
        totalRowVal += salesDF.at[i,j]
    # And then save into the Day Total column for that row.
    salesDF.at[i,"Day Total"] = totalRowVal
    
salesDF.head(5)

---
# Visualizing Data
Now, we can try visualizing the data using matplotlib (plt).
matplotlib and pandas play well with each other, and as a result, we can visualize things in multiple ways.
More advanced stuff can always be done with `matplotlib.pyplot`, which also supports DataFrames.

We'll showcase a few ways of visualizing our Seller-Day data, as well as some context to what these visualizations are typically suitable for.

**Histograms**<br/>
Histograms represent your data exactly as is based on a measure.<br/>
They look similar to bar graphs, but differ in that the data isn't 'categorized'.<br/>
Visually, this means the bars are connected to each other, as opposed to having some empty space between each other.<br/>

In [ ]:
salesDF.plot(kind='hist', x='Day', y='Day Total', figsize=(5,5))

# Another way to plot a histogram.
#salesDF[['Day Total']].hist(grid=False)

We can also use the 'Kernel Density Estimation' plot as another way to represent the distribution of data by probabilities.

In [ ]:
salesDF.plot(kind='kde', x='Day', y='Day Total', figsize=(8,8))

**Bar Graph**<br/>
We can use bar graphs to represent values in particular categories (qualitative data).<br/>
For example, we'll view the results of the first 10 days, and then the results of a particular seller.

In [ ]:
salesDFColumns = salesDF.columns.values;

# Total sales for the first 10 days. Let the graph be an 8in x 8in graph.
salesDF[:10].plot(kind='bar', x='Day', y='Day Total', figsize=(8,8))

In [ ]:
# https://matplotlib.org/api/axes_api.html?highlight=matplotlib%20axes%20axes#module-matplotlib.axes

# We can also turn it into a stacked bar-graph in order to see the breakdowns of each seller.
# You can also optionally store the result of plot in order to add axis labels.
salesBarA = salesDF[:10].plot(kind='bar', x='Day', y='Alex', figsize=(6,6))
salesBarA.set_xlabel("Days")
salesBarA.set_ylabel("Alex's Daily Sales")

# Hardcoded label. (Bad idea.)
#salesBarA.text(x=-0.2, y=330, s='320')

# Automated by annotating each bar (patch) in the graph.
# Needs alignment.
for x in salesBarA.patches:
    salesBarA.annotate(str(x.get_height()), (x.get_x(), x.get_height() * 1.020))

In [ ]:
salesDFColumns = salesDF.columns.values;

# Total sales for the first 10 days. Let the graph be a 6in x 6in stacked bar graph.
salesDF[:10].plot(kind='bar', x='Day Total', y=salesDFColumns[1:7], figsize=(6,6), rot=0, stacked=True)

In [ ]:
# Pie chart. We can also change the colour-palette using the 'colormap' parameter.
# Set1/2 sucks.
salesDF[:10].plot(kind='pie', title="First 10 Days of Sales", y='Day Total', figsize=(8,8), fontsize=15, colormap="tab10")

#https://matplotlib.org/users/colormaps.html

**Area/Density Graphs**<br/>
Area/Density graphs are great for quantitative/numbers-based data.

In [ ]:
# Similarly, we can also do a density/area plot. (Useful for quantitative/numbers-based data.)
# We can see that Denise and Alex made up the majority of sales on Day 10 here too.
salesDF[:10].plot(kind='area', x='Day', y=salesDFColumns[1:7], figsize=(8,8))

**Correlation-based Graphing**<br/>
While our data doesn't have any correlations (randomly generated), density and scatter plots are a good way to look for correlations between two variables.

In [ ]:
# Scatter plot.
salesDF.plot(kind='scatter', x='Day Total', y='Day', figsize=(8,8))

In [ ]:
# Hex Plot
salesDF.plot(kind='hexbin', x='Day', y='Day Total', gridsize=(20,20), figsize=(8,8), colormap='plasma')

**Line Graph (Spaghetti Plot)**<br/>
A spaghetti plot overlays multiple lines of sales.<br/>
Spaghetti plots are great to showcase performances between Sellers, but can easily become unreadable without proper effort done to highlight particular insights.

In [ ]:
# Matplotlib has multiple visual palettes/styles to use in order to change the way your graphs look.
# Seaborn is another visualizer with more aesthetically pleasing palettes.
print("Matplotlib Available Styles:\n",plt.style.available)
salesDF[:10].plot(figsize=(8,8), y=salesDFColumns[1:7])

In [ ]:
# Using matplotlib's example of multiple spaghetti graphs as a reference.
# https://python-graph-gallery.com/125-small-multiples-for-line-chart/
# https://matplotlib.org/gallery/color/colormap_reference.html
plt.style.use('seaborn-darkgrid') # Pick a style available through matplotlib to make out graph look nice.
palette = plt.get_cmap('tab10') # Pick the color set of lines.
salesDFShape = salesDF.shape # Get the number of rows/cols.
plt.figure(figsize=(10,8)) # Set the figure size


num = 0
for column in salesDFColumns[1:salesDFShape[1]-1]:
    # Set up the subplots.
    num += 1
    plt.subplot(2,3,num)   
    # Plot the shadows of everything else. Also make them thin and light so they don't get in the way.
    for v in salesDFColumns[1:salesDFShape[1]-1]:
        plt.plot(salesDF['Day'], salesDF[v], marker='', color='grey', linewidth=0.6, alpha=0.3)
    
    # Plot the coloured line, and set graph's x/y limits.
    plt.plot(salesDF['Day'], salesDF[column], marker='', linewidth=1.9, alpha=0.9, color=palette(num)) 
    plt.xlim(0,10) # 10 Days
    plt.ylim(0,1000) # Y-Axis that goes up to 1000 sales.
    plt.title(s=salesDFColumns[num], loc='left', fontsize=15, color=palette(num)) # Title the subplot with the seller's name.
    
    # Remove some of the x/y axis so that they're only in the far left/bottom.
    if num in range(4):
        plt.tick_params(labelbottom=False)
    if num not in [1,4,7]:
        plt.tick_params(labelleft=False)
        
# Super Title
plt.suptitle("Sales each Seller generated in the first 10 days.", fontsize=20, color='black', style='italic')
 
# Labeled Axis (Hard-coded.)
plt.text(-7, -150, 'Days', ha='center', va='center', fontsize=18)
plt.text(-26, 1100, 'Sales', ha='center', va='center', rotation='vertical', fontsize=18)

---
# Saving Your Work
Similar to importing files, `dataFrame.to_fileType("fileName.type")` exporting works for creating a product spreadsheet.

Graphs can be downloaded straight out of your notebook by saving the images.

Jupyter also supports exporting to a variety of filetypes like `.ipynb` notebooks, `.py` Python files, or `.html` HTML copies of your notebook.

In [ ]:
salesDF.to_csv("Result1.csv")